# LangGraph 2.4: Multi-Agent Architectures

## Complete Guide with Executable Examples

### Topics Covered:
1. **Collaboration vs Supervision Patterns**
2. **Shared Scratchpad Approach**
3. **Independent Agent Coordination**
4. **Agent-to-Agent Communication**
5. **State Machine Concepts**

## Setup and Installation

In [3]:
!pip install langgraph langchain-anthropic langchain-core -q

In [4]:
import os
from typing import Annotated, Literal, TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from datetime import datetime
import time

os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

print("✅ Imports completed!")

✅ Imports completed!


---

# 1. Collaboration vs Supervision Patterns

**Collaboration**: Agents work as peers
**Supervision**: Hierarchical with manager-worker model

## 1.1 Collaboration Pattern - Peer Review

In [7]:
class CollaborationState(TypedDict):
    messages: Annotated[list, add_messages]
    document: str
    reviews: list
    consensus_reached: bool
    final_version: str

def writer_agent(state):
    print("✍️  Writer: Creating draft...")
    return {
        "messages": [AIMessage(content="Draft created", name="Writer")],
        "document": "Initial technical documentation draft."
    }

def reviewer1_agent(state):
    print("👀 Reviewer 1: Analyzing...")
    review = {"agent": "R1", "feedback": "Needs more details", "rating": 7}
    reviews = state.get("reviews", []) + [review]
    print(f"   Rating: {review['rating']}/10")
    return {"messages": [AIMessage(content="Reviewed", name="R1")], "reviews": reviews}

def reviewer2_agent(state):
    print("👀 Reviewer 2: Analyzing...")
    review = {"agent": "R2", "feedback": "Add examples", "rating": 8}
    reviews = state.get("reviews", []) + [review]
    print(f"   Rating: {review['rating']}/10")
    return {"messages": [AIMessage(content="Reviewed", name="R2")], "reviews": reviews}

def check_consensus(state):
    reviews = state.get("reviews", [])
    if len(reviews) < 2:
        return {"consensus_reached": False}
    avg_rating = sum(r["rating"] for r in reviews) / len(reviews)
    consensus = avg_rating >= 7.5
    print(f"🔍 Avg rating: {avg_rating:.1f} - Consensus: {consensus}")
    return {"consensus_reached": consensus}

def route_consensus(state) -> Literal["revise", "finalize"]:
    return "finalize" if state.get("consensus_reached") else "revise"

def revise_agent(state):
    print("✍️  Writer: Revising...")
    feedback = "; ".join([r["feedback"] for r in state["reviews"]])
    return {"document": state["document"] + f" [REVISED: {feedback}]", "reviews": []}

def finalize(state):
    print("✅ Finalized!")
    return {"final_version": state["document"]}

# Build workflow
workflow = StateGraph(CollaborationState)
workflow.add_node("writer", writer_agent)
workflow.add_node("reviewer1", reviewer1_agent)
workflow.add_node("reviewer2", reviewer2_agent)
workflow.add_node("check", check_consensus)
workflow.add_node("revise", revise_agent)
workflow.add_node("finalize", finalize)

workflow.add_edge(START, "writer")
workflow.add_edge("writer", "reviewer1")
workflow.add_edge("writer", "reviewer2")
workflow.add_edge("reviewer1", "check")
workflow.add_edge("reviewer2", "check")
workflow.add_conditional_edges("check", route_consensus, {"revise": "revise", "finalize": "finalize"})
workflow.add_edge("revise", "reviewer1")
workflow.add_edge("revise", "reviewer2")
workflow.add_edge("finalize", END)

collab_app = workflow.compile()

# Test
result = collab_app.invoke({"messages": [], "document": "", "reviews": [], "consensus_reached": False, "final_version": ""})
print(f"\n📄 Final: {result['final_version']}")

✍️  Writer: Creating draft...
👀 Reviewer 1: Analyzing...
   Rating: 7/10
👀 Reviewer 2: Analyzing...
   Rating: 8/10


InvalidUpdateError: At key 'reviews': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_CONCURRENT_GRAPH_UPDATE

## 1.2 Supervision Pattern - Manager-Worker

In [14]:
class SupervisionState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    worker_results: dict

def supervisor(state):
    print("👔 Supervisor: Delegating tasks...")
    return {"messages": [AIMessage(content="Tasks assigned", name="Supervisor")]}

def researcher(state):
    print("🔬 Researcher: Working...")
    results = state.get("worker_results", {})
    results["research"] = "Research complete"
    return {"worker_results": results}

def coder(state):
    print("💻 Coder: Working...")
    results = state.get("worker_results", {})
    results["code"] = "Code complete"
    return {"worker_results": results}

def supervisor_review(state):
    print("👔 Supervisor: Reviewing...")
    print(f"   Results: {state['worker_results']}")
    return state

# Build
workflow = StateGraph(SupervisionState)
workflow.add_node("supervisor", supervisor)
workflow.add_node("researcher", researcher)
workflow.add_node("coder", coder)
workflow.add_node("review", supervisor_review)

workflow.add_edge(START, "supervisor")
workflow.add_edge("supervisor", "researcher")
workflow.add_edge("supervisor", "coder")
workflow.add_edge("researcher", "review")
workflow.add_edge("coder", "review")
workflow.add_edge("review", END)

super_app = workflow.compile()

# Test
result = super_app.invoke({"messages": [], "task": "Build feature", "worker_results": {}})
print(f"\n✅ Done")

👔 Supervisor: Delegating tasks...
💻 Coder: Working...
🔬 Researcher: Working...


InvalidUpdateError: At key 'worker_results': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_CONCURRENT_GRAPH_UPDATE

---

# 2. Shared Scratchpad Approach

All agents read/write to shared workspace

In [16]:
class ScratchpadState(TypedDict):
    messages: Annotated[list, add_messages]
    scratchpad: dict
    final_solution: str

def analyzer(state):
    print("🔍 Analyzer: Writing to scratchpad...")
    scratchpad = state.get("scratchpad", {})
    scratchpad["analysis"] = {"goal": "Optimize", "constraints": ["Memory", "Speed"]}
    print(f"   ✍️  Wrote: analysis")
    return {"scratchpad": scratchpad}

def researcher(state):
    print("🔬 Researcher: Reading & writing...")
    scratchpad = state.get("scratchpad", {})
    print(f"   📖 Read: {list(scratchpad.keys())}")
    scratchpad["research"] = {"approaches": ["Caching", "Async"]}
    print(f"   ✍️  Wrote: research")
    return {"scratchpad": scratchpad}

def architect(state):
    print("🏗️  Architect: Reading & designing...")
    scratchpad = state.get("scratchpad", {})
    print(f"   📖 Read: {list(scratchpad.keys())}")
    scratchpad["design"] = {"layers": ["Cache", "Queue", "DB"]}
    print(f"   ✍️  Wrote: design")
    return {"scratchpad": scratchpad}

def implementer(state):
    print("💻 Implementer: Reading scratchpad...")
    scratchpad = state.get("scratchpad", {})
    print(f"   📖 Read all: {list(scratchpad.keys())}")
    solution = f"Solution based on: {list(scratchpad.keys())}"
    return {"final_solution": solution}

# Build
workflow = StateGraph(ScratchpadState)
workflow.add_node("analyzer", analyzer)
workflow.add_node("researcher", researcher)
workflow.add_node("architect", architect)
workflow.add_node("implementer", implementer)

workflow.add_edge(START, "analyzer")
workflow.add_edge("analyzer", "researcher")
workflow.add_edge("researcher", "architect")
workflow.add_edge("architect", "implementer")
workflow.add_edge("implementer", END)

scratchpad_app = workflow.compile()

# Test
result = scratchpad_app.invoke({"messages": [], "scratchpad": {}, "final_solution": ""})
print(f"\n📋 Scratchpad: {list(result['scratchpad'].keys())}")
print(f"📄 Solution: {result['final_solution']}")

🔍 Analyzer: Writing to scratchpad...
   ✍️  Wrote: analysis
🔬 Researcher: Reading & writing...
   📖 Read: ['analysis']
   ✍️  Wrote: research
🏗️  Architect: Reading & designing...
   📖 Read: ['analysis', 'research']
   ✍️  Wrote: design
💻 Implementer: Reading scratchpad...
   📖 Read all: ['analysis', 'research', 'design']

📋 Scratchpad: ['analysis', 'research', 'design']
📄 Solution: Solution based on: ['analysis', 'research', 'design']


---

# 3. Independent Agent Coordination

Agents work independently but coordinate through dependencies

In [18]:
class CoordState(TypedDict):
    messages: Annotated[list, add_messages]
    resources: dict
    statuses: dict

def collector(state):
    print("📊 Collector: Working...")
    time.sleep(0.2)
    resources = state.get("resources", {})
    resources["data"] = {"records": 1000}
    statuses = state.get("statuses", {})
    statuses["collector"] = "done"
    print("   ✅ Data collected")
    return {"resources": resources, "statuses": statuses}

def processor(state):
    print("⚙️  Processor: Checking dependencies...")
    if "data" not in state.get("resources", {}):
        print("   ⏸️  Blocked - waiting for data")
        statuses = state.get("statuses", {})
        statuses["processor"] = "blocked"
        return {"statuses": statuses}
    
    print("   ✅ Processing...")
    time.sleep(0.2)
    resources = state.get("resources", {})
    resources["processed"] = True
    statuses = state.get("statuses", {})
    statuses["processor"] = "done"
    return {"resources": resources, "statuses": statuses}

def reporter(state):
    print("📝 Reporter: Checking dependencies...")
    if not state.get("resources", {}).get("processed"):
        print("   ⏸️  Blocked - waiting for processing")
        statuses = state.get("statuses", {})
        statuses["reporter"] = "blocked"
        return {"statuses": statuses}
    
    print("   ✅ Generating report...")
    time.sleep(0.2)
    statuses = state.get("statuses", {})
    statuses["reporter"] = "done"
    return {"statuses": statuses}

# Build
workflow = StateGraph(CoordState)
workflow.add_node("collector", collector)
workflow.add_node("processor", processor)
workflow.add_node("reporter", reporter)

workflow.add_edge(START, "collector")
workflow.add_edge(START, "processor")
workflow.add_edge(START, "reporter")
workflow.add_edge("collector", END)
workflow.add_edge("processor", END)
workflow.add_edge("reporter", END)

coord_app = workflow.compile()

# Test
result = coord_app.invoke({"messages": [], "resources": {}, "statuses": {}})
print(f"\n📊 Statuses: {result['statuses']}")

📊 Collector: Working...
⚙️  Processor: Checking dependencies...
   ⏸️  Blocked - waiting for data
📝 Reporter: Checking dependencies...
   ⏸️  Blocked - waiting for processing
   ✅ Data collected


InvalidUpdateError: At key 'statuses': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_CONCURRENT_GRAPH_UPDATE

---

# 4. Agent-to-Agent Communication

Direct messaging between agents

In [20]:
class CommState(TypedDict):
    messages: Annotated[list, add_messages]
    inbox: dict
    history: list

def agent_a_send(state):
    print("🤖 Agent A: Sending message...")
    msg = {"from": "A", "to": "B", "content": "Process [1,2,3]"}
    inbox = state.get("inbox", {})
    inbox.setdefault("B", []).append(msg)
    history = state.get("history", []) + [msg]
    print(f"   📤 Sent to B")
    return {"inbox": inbox, "history": history}

def agent_b_receive(state):
    print("🤖 Agent B: Checking inbox...")
    inbox = state.get("inbox", {})
    if not inbox.get("B"):
        print("   📭 Empty")
        return state
    
    msg = inbox["B"][-1]
    print(f"   📬 Received: {msg['content']}")
    
    response = {"from": "B", "to": "A", "content": "Processed: sum=6"}
    inbox.setdefault("A", []).append(response)
    history = state.get("history", []) + [response]
    print(f"   📤 Responded")
    return {"inbox": inbox, "history": history}

# Build
workflow = StateGraph(CommState)
workflow.add_node("a_send", agent_a_send)
workflow.add_node("b_receive", agent_b_receive)

workflow.add_edge(START, "a_send")
workflow.add_edge("a_send", "b_receive")
workflow.add_edge("b_receive", END)

comm_app = workflow.compile()

# Test
result = comm_app.invoke({"messages": [], "inbox": {}, "history": []})
print(f"\n💬 History:")
for msg in result["history"]:
    print(f"   {msg['from']} → {msg['to']}: {msg['content']}")

🤖 Agent A: Sending message...
   📤 Sent to B
🤖 Agent B: Checking inbox...
   📬 Received: Process [1,2,3]
   📤 Responded

💬 History:
   A → B: Process [1,2,3]
   B → A: Processed: sum=6


---

# 5. State Machine Concepts

Well-defined states and transitions

In [22]:
class OrderState(TypedDict):
    messages: Annotated[list, add_messages]
    current_state: str
    order_id: str
    order_data: dict

def pending(state):
    print(f"📦 STATE: PENDING")
    data = state.get("order_data", {})
    data["status"] = "pending"
    return {"current_state": "PENDING", "order_data": data}

def validating(state):
    print(f"✓ STATE: VALIDATING")
    data = state.get("order_data", {})
    data["validated"] = True
    return {"current_state": "VALIDATING", "order_data": data}

def processing(state):
    print(f"⚙️  STATE: PROCESSING")
    data = state.get("order_data", {})
    data["processed"] = True
    return {"current_state": "PROCESSING", "order_data": data}

def completed(state):
    print(f"✅ STATE: COMPLETED")
    data = state.get("order_data", {})
    data["completed"] = True
    return {"current_state": "COMPLETED", "order_data": data}

def transition(state) -> Literal["validating", "processing", "completed"]:
    current = state.get("current_state", "PENDING")
    if current == "PENDING":
        print("🔀 PENDING → VALIDATING")
        return "validating"
    elif current == "VALIDATING":
        print("🔀 VALIDATING → PROCESSING")
        return "processing"
    elif current == "PROCESSING":
        print("🔀 PROCESSING → COMPLETED")
        return "completed"
    return "completed"

# Build
workflow = StateGraph(OrderState)
workflow.add_node("pending", pending)
workflow.add_node("validating", validating)
workflow.add_node("processing", processing)
workflow.add_node("completed", completed)

workflow.add_edge(START, "pending")
workflow.add_conditional_edges("pending", transition, {"validating": "validating"})
workflow.add_conditional_edges("validating", transition, {"processing": "processing"})
workflow.add_conditional_edges("processing", transition, {"completed": "completed"})
workflow.add_edge("completed", END)

sm_app = workflow.compile()

# Test
result = sm_app.invoke({"messages": [], "current_state": "PENDING", "order_id": "ORD-123", "order_data": {}})
print(f"\n📦 Final state: {result['current_state']}")
print(f"📊 Order data: {result['order_data']}")

📦 STATE: PENDING
🔀 PENDING → VALIDATING
✓ STATE: VALIDATING
🔀 VALIDATING → PROCESSING
⚙️  STATE: PROCESSING
🔀 PROCESSING → COMPLETED
✅ STATE: COMPLETED

📦 Final state: COMPLETED
📊 Order data: {'status': 'pending', 'validated': True, 'processed': True, 'completed': True}


---

# Summary

## Pattern Selection Guide:

**Use COLLABORATION when:**
- Agents are peers with equal expertise
- Need consensus or peer review
- Building shared understanding

**Use SUPERVISION when:**
- Clear hierarchy needed
- Central coordination required
- Delegating distinct tasks

**Use SCRATCHPAD when:**
- Incremental problem solving
- Building on shared knowledge
- Sequential agent contributions

**Use COMMUNICATION when:**
- Agents need to negotiate
- Information exchange required
- Request-response patterns

**Use STATE MACHINE when:**
- Clear process stages
- Defined lifecycle
- Event-driven workflows

## Best Practices:

✅ Define clear agent responsibilities

✅ Minimize shared state

✅ Handle communication failures

✅ Track agent progress

✅ Log state transitions